# Parametrization of a simulation algorithm

In this example we are going to parameterize a simulation algorithm:

 - parameters linked to the number of points generated
 - the precision of the probability estimator
 - the sample storage strategy
 - using callbacks to monitor progress and stopping criteria.
 

In [1]:
from __future__ import print_function
import openturns as ot

Create the joint distribution of the parameters.

In [2]:
distribution_R = ot.LogNormalMuSigma(300.0, 30.0, 0.0).getDistribution()
distribution_F = ot.Normal(75e3, 5e3)
marginals = [distribution_R, distribution_F]
distribution = ot.ComposedDistribution(marginals)

Create the model.

In [3]:
model = ot.SymbolicFunction(['R', 'F'], ['R-F/(pi_*100.0)'])

Create the event whose probability we want to estimate.

In [4]:
vect = ot.RandomVector(distribution)
G = ot.CompositeRandomVector(model, vect)
event = ot.ThresholdEvent(G, ot.Less(), 0.0)

Create a Monte Carlo algorithm.

In [5]:
experiment = ot.MonteCarloExperiment()
algo = ot.ProbabilitySimulationAlgorithm(event, experiment)

Criteria 1: Define the Maximum Coefficient of variation of the probability estimator. 

In [6]:
algo.setMaximumCoefficientOfVariation(0.05)

Criteria 2: Define the number of iterations of the simulation.

In [7]:
algo.setMaximumOuterSampling(int(1e4))

The block size parameter represents the number of samples evaluated per iteration, useful for parallelization. 

In [8]:
algo.setBlockSize(2)

HistoryStrategy to store the values of the probability used to draw the convergence graph. 

In [9]:
# Null strategy
algo.setConvergenceStrategy(ot.Null())

# Full strategy
algo.setConvergenceStrategy(ot.Full())

# Compact strategy: N points
N = 1000
algo.setConvergenceStrategy(ot.Compact(N))

Use a callback to display the progress every 10%. 

In [10]:
def progress(p):
    if p >= progress.t:
        progress.t += 10.0
        print('progress=', p, '%')
    return False
progress.t = 10.0
algo.setProgressCallback(progress)

Use a callback to stop the simulation. 

In [11]:
def stop():
    # here we never stop, but we could
    return False
algo.setStopCallback(stop)

In [12]:
algo.run()

progress= 10.0 %
progress= 20.0 %
progress= 30.0 %
progress= 40.0 %
progress= 50.0 %
progress= 60.0 %
progress= 70.0 %
progress= 80.0 %
progress= 90.0 %
progress= 100.0 %


Retrieve results. 

In [13]:
result = algo.getResult()
probability = result.getProbabilityEstimate()
print('Pf=', probability)

Pf= 0.02859999999999999
